In [1]:
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

# from torch.linalg import svd
from torch.linalg import svd

from torch.nn.functional import relu

from emb_vectors_functions import (
    find_self_embeds,
    get_model_and_embed,
    get_shadow_ratios,
)

In [2]:
model_name = "meta-llama/Llama-3.2-1B"  # "meta-llama/Meta-Llama-3.1-8B"  # "meta-llama/Meta-Llama-3.1-8B" , "gpt2", "meta-llama/Llama-2-7b-hf"
model, embedding, head, model_norm, mean_norm, tokenizer = get_model_and_embed(
    model_name
)

del model, embedding
embeddings = head
embeddings.requires_grad = False

In [3]:
fail_indices, failed_res_emb, failed_pairs = find_self_embeds(embeddings, tokenizer)

100%|██████████| 13/13 [00:02<00:00,  6.26it/s]


In [4]:
shadow_ratios = get_shadow_ratios(fail_indices, embeddings)
shadow_ratios_sorted = sorted(shadow_ratios, key=lambda x: x[1], reverse=True);

In [114]:
n = 177
embeddings_cut = embeddings  # embeddings[:10000]
embeddings_others = torch.cat((embeddings_cut[:n], embeddings_cut[n + 1 :]), dim=0)
self_emb = embeddings_cut[n]

A = self_emb - embeddings_others
A.requires_grad = False
torch.all(A @ self_emb > 0)

# embeddings_others = embeddings_others.detach().cpu()
# self_emb = self_embed.detach().cpu()
# A = A.detach().cpu()

tensor(False)

In [115]:
x = self_emb.detach().clone()
x.requires_grad = True
optimizer = torch.optim.SGD([x], lr=0.01)
optimizer = torch.optim.AdamW([x], lr=0.01)

In [116]:
epsilon = 1e-4

with torch.no_grad():
    loss = torch.sum(relu(-A @ x + epsilon))
    print(f"Initial loss = {loss}")

for step in range(1000):
    optimizer.zero_grad()
    loss = torch.sum(relu(-A @ x + epsilon))
    loss.backward()
    optimizer.step()

    if (step + 1) % 10 == 0:
        print(f"Step {step+1}, Loss: {loss.item()}")
    if loss.item() <= 0:
        break

print("Optimization finished. Final x:", loss.item())
print(f"Target embedding - {torch.all(A@x>0).item()}, min = {torch.min(A@x)}")

Initial loss = 6.931521415710449
Step 10, Loss: 0.030173495411872864
Step 20, Loss: 0.02480473183095455
Step 30, Loss: 0.017095521092414856
Step 40, Loss: 0.009619321674108505
Step 50, Loss: 0.004163955803960562
Step 60, Loss: 0.0018103988841176033
Step 70, Loss: 0.0009649852872826159
Step 80, Loss: 0.0005561368889175355
Step 90, Loss: 0.00037754763616248965
Step 100, Loss: 0.0003059612645301968
Step 110, Loss: 0.0002523293369449675
Step 120, Loss: 0.0002088374167215079
Step 130, Loss: 0.0001777974102878943
Step 140, Loss: 0.00015200275811366737
Step 150, Loss: 0.00013408041559159756
Step 160, Loss: 0.00012149220128776506
Step 170, Loss: 0.00010237032256554812
Step 180, Loss: 8.380766666959971e-05
Step 190, Loss: 6.539615424117073e-05
Step 200, Loss: 5.5209893616847694e-05
Step 210, Loss: 5.110912024974823e-05
Step 220, Loss: 4.761807213071734e-05
Step 230, Loss: 4.4579523091670126e-05
Step 240, Loss: 4.1886385588441044e-05
Step 250, Loss: 3.971511614508927e-05
Step 260, Loss: 3.761104

In [8]:
from scipy.optimize import linprog

In [ ]:
n, d = A.shape  # A is (n, d), x is (n,)
epsilon = 1e-6

c = np.zeros(d)  # No objective function, just feasibility
bounds = [(None, None)] * d  # x can take any value

res = linprog(c, A_ub=-A, b_ub=-epsilon * np.ones(n), bounds=bounds, method="highs")

In [22]:
# from numpy.linalg import svd
# from scipy.linalg import svd
from torch.linalg import svd

In [67]:
# U, S, Vt = svd(A.T, full_matrices=True)
U, S, Vt = svd(A.T)

In [68]:
# Null space basis vectors correspond to zero singular values
tol = 1e-10  # Tolerance to detect numerical zeros
null_space = Vt[S < tol]  # Extract null space basis

IndexError: The shape of the mask [2048] at index 0 does not match the shape of the indexed tensor [9999, 9999] at index 0

In [70]:
torch.linalg.matrix_rank(A.T)

tensor(2048)

In [73]:
import numpy as np
from scipy.optimize import linprog

In [86]:
d, n = A.T.shape
epsilon = 1e-6

c = np.zeros(n)
bounds = [(epsilon, None)] * n  # Enforce strict positivity (x_i > epsilon)
res = linprog(c, A_eq=A.T, b_eq=np.zeros(d), bounds=bounds, method="highs")

In [78]:
qq = res

In [83]:
res

       message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is Infeasible)
       success: False
        status: 2
           fun: None
             x: None
           nit: 69
         lower:  residual: None
                marginals: None
         upper:  residual: None
                marginals: None
         eqlin:  residual: None
                marginals: None
       ineqlin:  residual: None
                marginals: None